In [4]:
import pandas as pd
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

reader = Reader()
ratings = pd.read_csv('MovieLens_data/ratings_cleaned.csv')
print(ratings.head())

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

   userId  movieId  rating
0       1        2     3.5
1       1       29     3.5
2       1       32     3.5
3       1       47     3.5
4       1       50     3.5


## SVD Model

In [7]:
algorithm = SVD()

cross_validate(algorithm, data, measures=['RMSE', 'MAE'], cv=5, verbose=3)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7860  0.7868  0.7862  0.7864  0.7858  0.7862  0.0003  
MAE (testset)     0.5981  0.5988  0.5983  0.5984  0.5981  0.5983  0.0002  
Fit time          191.08  189.23  197.28  196.55  213.84  197.60  8.69    
Test time         72.29   64.83   59.62   73.61   78.39   69.75   6.68    


{'test_rmse': array([0.78603921, 0.78677499, 0.78617186, 0.78637742, 0.78581697]),
 'test_mae': array([0.59809748, 0.5987828 , 0.59828539, 0.59841097, 0.59813032]),
 'fit_time': (191.0799880027771,
  189.22729969024658,
  197.28398656845093,
  196.54926300048828,
  213.84378337860107),
 'test_time': (72.2936065196991,
  64.82668781280518,
  59.624117851257324,
  73.61310696601868,
  78.39264678955078)}

In [9]:
# test the model
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.25)
algorithm.fit(trainset)
predictions = algorithm.test(testset)

from surprise import accuracy
accuracy.rmse(predictions)
print(predictions[:5])

RMSE: 0.7892
[Prediction(uid=31061, iid=339, r_ui=4.0, est=4.515228803886983, details={'was_impossible': False}), Prediction(uid=84826, iid=930, r_ui=4.0, est=3.7765532367010817, details={'was_impossible': False}), Prediction(uid=110010, iid=2990, r_ui=3.5, est=3.413634143543167, details={'was_impossible': False}), Prediction(uid=95699, iid=1270, r_ui=3.5, est=3.7075207865260853, details={'was_impossible': False}), Prediction(uid=102737, iid=339, r_ui=5.0, est=3.70449313021564, details={'was_impossible': False})]


In [15]:
import pandas as pd

# Read the ratings data
ratings = pd.read_csv('MovieLens_data/ratings.csv')

# Check if 'timestamp' column exists
if 'timestamp' in ratings.columns:
    # Read the movies data
    movies = pd.read_csv('MovieLens_data/movies.csv')

    # Get the movie with the highest timestamp
    highest_timestamp_movie_id = ratings.loc[ratings['timestamp'].idxmax(), 'movieId']
    highest_timestamp_movie_title = movies.loc[movies['movieId'] == highest_timestamp_movie_id, 'title'].values[0]

    print(highest_timestamp_movie_title + " has the highest timestamp, with a value of " + str(ratings['timestamp'].max()))
else:
    print("The 'timestamp' column is missing in the ratings data.")

Girl with a Pearl Earring (2003) has the highest timestamp, with a value of 1427784002
